In [1]:
import os
import math
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pyproj import Proj
from shapely.geometry import LineString, Polygon  
import geopandas as gpd
from datetime import timedelta
from datetime import datetime


In [2]:
df = pd.read_csv('/glade/p/cisl/aiml/conv_risk_intel/spc_tornado_tracks/1950-2019_actual_tornadoes.csv')
df = df.loc[df['tz'] == 3]
print(f"Original df contains {df.shape[0]:,} tornado tracks")

df = df[df['yr']>2006].dropna()
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df = df.sort_values(by="datetime")
print(f"DF for years 2007 and after contains {df.shape[0]:,} tornado tracks")
print(df.columns)


Original df contains 65,125 tornado tracks
DF for years 2007 and after contains 15,866 tornado tracks
Index(['om', 'yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'stf', 'stn', 'mag',
       'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len',
       'wid', 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc', 'datetime'],
      dtype='object')


In [3]:
proj = Proj(proj='lcc', R=6371229, lat_0=25, lon_0=265, lat_1=25, lat_2=25)

df[['slon_m', 'slat_m']] = np.array(proj(list(df['slon']), list(df['slat']))).T
df[['elon_m', 'elat_m']] = np.array(proj(list(df['elon']), list(df['elat']))).T


In [4]:
def getEndpoint(d, begin_lon, begin_lat):
    
    d = d / 0.62137 #Convert d to km from miles
    R = 6371.229 #Radius of the Earth 
    brng = 45 * math.pi/180 #Bearing is 45 degrees converted to radians.

    begin_lon = math.radians(begin_lon)
    begin_lat = math.radians(begin_lat)

    end_lat = math.asin(math.sin(begin_lat)*math.cos(d/R) +
                        math.cos(begin_lat)*math.sin(d/R)*math.cos(brng))
    
    end_lon = begin_lon + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(begin_lat),
                                math.cos(d/R)-math.sin(begin_lat)*math.sin(end_lat))
    
    end_lat = math.degrees(end_lat)
    end_lon = math.degrees(end_lon)
    
    return end_lon, end_lat


In [5]:
for index, row in df.iterrows():
    if (df.index[0] - index) % 5000 == 0:
        print(f"Finished with {abs(df.index[0] - index)} rows")
    if np.isinf(row['slon_m']):
        print(f" - No begin lat lon at index {index}")
        continue
    if pd.isna(row['elon']):
        end_lon, end_lat = getEndpoint(row['len'], row['slon'], row['slat'])
        end_lon, end_lat = np.array(proj(end_lon, end_lat)).T
        line = LineString([[row['slon_m'], row['slat_m']],
                           [end_lon, end_lat]])
        rectangle = line.buffer(row['wid'] * 0.9144)       # yard to meter 
    else:
        line = LineString([[row['slon_m'], row['slat_m']],
                           [row['elon_m'], row['elat_m']]])
        rectangle = line.buffer(row['wid'] * 0.9144)       # yard to meter 
    end_lon, end_lat = rectangle.minimum_rotated_rectangle.exterior.coords.xy
    rectangle = Polygon(np.array(proj(end_lon, end_lat, inverse=True)).T)
    df.at[index, "track_rectangle"] = rectangle
print(f"There are {df.isnull().sum().sum()} NaN values in this df")
gdf = gpd.GeoDataFrame(df.reset_index(drop=True), geometry="track_rectangle")
gdf.to_file(f"/glade/p/cisl/aiml/jtti_tornado/track_rectangles_spc/track_rectangles_all.geojson", driver='GeoJSON')


Finished with 0 rows


KeyboardInterrupt: 

In [ ]:
def bin_12Z(row):
    if row['datetime'].hour < 12:
        return pd.to_datetime(row['datetime'].date() - timedelta(days=1)) + timedelta(hours=12)
    else:
        return pd.to_datetime(row['datetime'].date()) + timedelta(hours=12)

df['12Z'] = df.apply(bin_12Z, axis=1)


In [ ]:
for name, group in df.groupby('12Z'):
    if group.shape[0] == 0:
        print(f"DF at datetime {name} of size 0")
        continue
    else:
        date = name.strftime('%Y%m%d') 
        gdf = gpd.GeoDataFrame(group.reset_index(drop=True), geometry="track_rectangle")
        gdf.to_file(f"/glade/p/cisl/aiml/jtti_tornado/track_rectangles_spc/track_rectangles_{date}.geojson", driver='GeoJSON')
    